# Interpreting One-Hot Encoded Coefficients

## Introduction

You have already seen some examples of how to interpret coefficients for multiple linear regression. In this lesson we will go over some more examples, particularly focusing on models with one-hot encoded categorical predictors.

## Objectives

You will be able to:

* Describe and apply the concept of a reference category
* Interpret coefficients for one-hot encoded variables
* Explain the implications of binning and dropping multiple categories with one-hot encoded variables

## Reference Categories

Let's look at the Auto MPG dataset, with an engineered `make` feature. Then we'll start with a multiple regression model that uses `weight`, `model year`, and `origin` to predict `mpg`.

In [1]:
import pandas as pd

# Load data into pandas and engineer "make" feature
data = pd.read_csv("auto-mpg.csv")
data["make"] = data["car name"].str.split().apply(lambda x: x[0])

data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,make
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320,buick
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite,plymouth
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst,amc
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino,ford
...,...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl,ford
388,44.0,4,97.0,52,2130,24.6,82,2,vw pickup,vw
389,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage,dodge
390,28.0,4,120.0,79,2625,18.6,82,1,ford ranger,ford


In [2]:
# Prepare y and X for modeling
y = data["mpg"]
X = data[["weight", "model year", "origin"]]

# origin is categorical and needs to be numeric to run regression
X = pd.get_dummies(X, columns=["origin"], drop_first=True, dtype=int) 

In [ ]:
import statsmodels.api as sm

model = sm.OLS(y,X)
results = model.fit()

print(results.summary())

In the model displayed above, we have dropped one of the dummy variable columns for `origin`, in order to avoid the **dummy variable trap**. The dropped column is now our **reference category**.

So, which column is that?

To investigate this, we need to look at the values of `origin` prior to one-hot encoding:

In [ ]:
data["origin"].value_counts()

(`1` means a US car maker, `2` means a European car maker, `3` means an Asian car maker.)

Because `1` does not appear in our one-hot encoded variable (we only have `origin_2` and `origin_3`), we know that **`1` is the reference category**. Thus we can interpret the coefficients of `origin_2` and `origin_3` like this:

* `origin_2` means the difference associated with a car being from a European car maker vs. a US car maker. In other words, compared to US car makers, we see an associated increase of about 2 MPG for European car makers.
* `origin_3` means the difference associated with a car being from an Asian car maker vs. a US car maker. We see an associated increase of about 2.2 MPG for Asian car makers compared to US car makers.

### Alternative Reference Categories

When you run `pd.get_dummies` and specify `drop_first=True`, this is an easy way to make sure that you avoid the dummy variable trap and that _some_ category is selected as the reference category. However there is no data understanding involved; pandas just chooses the first category (alphabetically) that it finds.

If we skip that argument and manually drop a column instead, we can easily set any category as the reference category.

For example, if we set `2` (European car maker) as the reference category, our model looks like this:

In [ ]:
X = data[["weight", "model year", "origin"]]
X = pd.get_dummies(X, columns=["origin"], dtype=int)
X = X.drop("origin_2", axis=1)

In [ ]:
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

results.params

Or if we set `3` (Asian car maker) as the reference category, our model looks like this:

In [ ]:
X = data[["weight", "model year", "origin"]]
X = pd.get_dummies(X, columns=["origin"], dtype=int)
X = X.drop("origin_3", axis=1)

In [ ]:
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

results.params

Looking at the models above, what changes and what stays the same, in terms of the coefficients? Why?

---

<details>
    <summary style="cursor: pointer"><b>Answer (click to reveal)</b></summary>
    
Each time the reference category changes, the `const` and the other `origin` coefficients change.

`const` changes because it represents the value when _all_ predictors are 0, and this means that `const` represents when the reference category is true.

The other `origin` coefficients change because they are always with respect to the reference category. So if the category `1` has been dropped, then `origin_2` is with respect to category `1`. But if category `3` has been dropped instead, then `origin_2` is with respect to category `3`.

`weight` and `model year` don't change, because the `origin` overall is still being "held constant" by the model, regardless of how `origin` is actually encoded.

</details>

### Choosing a Reference Category

Choosing an appropriate reference category comes down to your understanding of the data and the stakeholders. Is there a category that makes the most sense as a "baseline", "default", or "neutral" category?

For our `origin` category, if you were performing this analysis in the US, then setting `origin_1` as the baseline might make the most sense. If your audience were in Europe or Asia, then it might make more sense to drop `origin_2` or `origin_3`. For a context without a clear match like this, you might choose the category that is most familiar or most well-known to your audience.

You also might consider looking at the data itself to see what makes the most sense as a "baseline". If one category is much more common than the others, that might be a good baseline. Or if one category is in the "middle" (maybe closest to the mean or median) in terms of the target, that also might be a good baseline.

**There is no single right answer for choosing a reference category.** The model contains the same information regardless of which category is chosen, so you are really just deciding how best to interpret your coefficients.

## Predictors with More Categories

The category `origin` only has three options for a reference category, since there are only three different values present in that column.

What about `make`, which contains many more categories?

In [ ]:
pd.get_dummies(data["make"], dtype=int)

We could just use `drop_first=True`, add this predictor to our other three, and see what the model outputs:

In [ ]:
X = data[["weight", "model year", "origin", "make"]]
X = pd.get_dummies(X, columns=["origin", "make"], drop_first=True, dtype=int)
X

In [ ]:
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

print(results.summary())

What a long model summary! How can we make sense of this, particularly given that so many of these coefficients are not statistically significant?

It's difficult to extract much meaning from the `make` variable here.

### Data Cleaning

Before doing anything else with categorical data, some ***data cleaning*** is often useful.

Something you might notice here is that several of these different `make` values are "really" the same. For example, `"chevroelt"` is almost certainly a typo, that was intended to be `"chevrolet"`.

What happens if we combine together values that seem like they really should be the same?

In [ ]:
print(data["make"].nunique(), "unique values")
data["make"].value_counts()

In [ ]:
replacement_dict = {
    "chevroelt": "chevrolet",
    "mercedes": "mercedes-benz",
    "toyouta": "toyota",
    "vokswagen": "volkswagen",
    "maxda": "mazda",
    "chevy": "chevrolet",
    "vw": "volkswagen"
}

data.replace(replacement_dict, inplace=True)

print(data["make"].nunique(), "unique values")
data["make"].value_counts()

In [ ]:
X = data[["weight", "model year", "origin", "make"]]
X = pd.get_dummies(X, columns=["origin", "make"], drop_first=True, dtype=int)
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

print(results.summary())

Well, that didn't do much. We still have lots of coefficients that can't be interpreted because they aren't statistically significant.

### Creating an "Other" Category

Another approach you might consider is creating a catch-all "other" category for categories that don't have many examples in the dataset. We'll choose a somewhat-arbitrary cutoff of **10 observations**, which is a conventional cutoff for the number of samples you need per independent variable.

In [ ]:
data["make"].value_counts()[data["make"].value_counts() < 10]

In [ ]:
to_replace = data["make"].value_counts()[data["make"].value_counts() < 10].index.values
to_replace

In [ ]:
data.replace(to_replace, value="other", inplace=True)

print(data["make"].nunique(), "unique values")
data["make"].value_counts()

In [ ]:
X = data[["weight", "model year", "origin", "make"]]
X = pd.get_dummies(X, columns=["origin", "make"], drop_first=True, dtype=int)
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

print(results.summary())

That is a much more manageable number of coefficients. Let's go through and interpret these:

* The **reference category** for `origin` is `1` (US) and for `make` is `amc` (American Motor Company)
* `const`, `weight`, and `model year` are all still statistically significant
  * When all other predictors are 0, the MPG would be about -18.3
  * For each increase of 1 lb in weight, we see an associated decrease of about 0.006 in MPG
  * For each year newer the vehicle is, we see an associated increase of about 0.75 in MPG
* `origin_2` and `origin_3` are not statistically significant any more
  * While this might seem surprising, our data understanding can explain it. The `origin` feature and the `make` feature are really providing the same information, except that `make` is more granular. Every `make` category (except for `other`) corresponds to exactly one `origin` category. Therefore it probably does not make sense to include both `origin` and `make` in the same model
* At a standard alpha of 0.05, only `make_plymouth`, `make_pontiac`, and `make_volkswagen` are statistically significant
  * When a car's make is `plymouth` compared to `amc`, we see an associated increase of about 2.4 in MPG
  * When a car's make is `pontiac` compared to `amc`, we see an associated increase of about 2.9 in MPG
  * When a car's make is `volkswagen` compared to `amc`, we see an associated increase of about 3.1 in MPG

All of the significant coefficients happen to be positive. Why is that? It turns out that `amc` is the first `make` value alphabetically _and_ has the lowest mean MPG:

In [ ]:
data.groupby("make").mean('mpg').sort_values(by="mpg")[["mpg"]]

It might surprise you that out of these categories, `pontiac`, `plymouth`, and `volkswagen` were the only ones with statistically significant differences. You wouldn't guess that based on the means alone. But keep in mind that the t-test being used here takes into account the number and the variance in each category, not just the mean.

### Model Iteration

We could stop there and have this be our final model. Coefficients that aren't statistically significant aren't a _problem_ for a model, they just mean that we can't interpret those coefficients because we don't have enough evidence that those coefficients aren't really zero.

A lot of people who are just learning about linear regression modeling will try dropping all of the predictors whose coefficients aren't statistically significant:

In [ ]:
X = data[["weight", "model year", "origin", "make"]]
X = pd.get_dummies(X, columns=["origin", "make"], drop_first=True, dtype=int)

not_significant = [
    "origin_2",
    "origin_3",
    "make_buick",
    "make_chevrolet",
    "make_datsun",
    "make_dodge",
    "make_ford",
    "make_honda",
    "make_mazda",
    "make_mercury",
    "make_oldsmobile",
    "make_other",
    "make_toyota"
]

X.drop(not_significant, axis=1, inplace=True)

model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

print(results.summary())

**Don't do this!**

First, what do these coefficients mean in this context? There are three different `model` values, so...

* `make_plymouth` is the difference between having a `make` of `plymouth` and a `make` of...anything other than `plymouth`, `pontiac`, or `volkswagen`
* `make_pontiac` is the difference between having a `make` of `pontiac` and a `make` of anything other than `plymouth`, `pontiac`, or `volkswagen`
* `make_volkswagen` is the difference between having a `make` of `volkswagen` and a `make` of anything other than `plymouth`, `pontiac`, or `volkswagen`

That "anything other than" category is no longer a useful reference category!

Second, you did not actually solve the statistical significance problem by dropping all of those columns. Now that those other columns are gone, `make_plymouth` and `make_pontiac` are no longer statistically significant. This is because even though there was a statistically significant difference between `plymouth` and `amc`, and between `pontiac` and `amc`, the difference between these categories and the "anything other than" category is not statistically significant.

A better strategy would go back to our **data understanding** and remove columns that we have a good reason to believe are no longer useful. In this case that would be the `origin` columns, since `make` is already providing that same information.

In [ ]:
X = data[["weight", "model year", "make"]]
X = pd.get_dummies(X, columns=["make"], drop_first=True, dtype=int)

model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

print(results.summary())

Because we made our modeling decision based on data understanding and not on p-values alone, we actually see that _more_ of our coefficients are significant than before.

Let's go ahead and say that this is our final model.

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y, results.predict(sm.add_constant(X)), squared=False)

## Final Model Results

Here we'll walk through a **narrative** version of our final model, starting from the beginning. It is fairly typical that your actual modeling process will not match the narrative you describe at the end, since the final version will be cleaned up and won't include extraneous details about things you tried that didn't work. For example, unless you had a specific reason to tell stakeholders why `origin` wasn't part of the final model, your final narrative can just skip over talking about `origin` at all, as this one does.

### Data Understanding and Preparation

Our final model included these features:

* `weight`
* `model year`
* `make`

We performed data cleaning on `make` to account for typos and alternative names, and also combined vehicles with fewer than 10 samples for their make into a single category called `other`. Then we one-hot encoded `make`, resulting in 14 dummy predictors.

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,5), sharey=True)
ax1.set_ylabel("mpg")

data.plot.scatter(x="weight", y="mpg", ax=ax1)
data.plot.scatter(x="model year", y="mpg", ax=ax2);

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
data.groupby("make").mean('mpg').sort_values(by="mpg").plot.bar(y="mpg", ax=ax)
ax.axhline(y=data["mpg"].mean(), label="mean", color="black", linestyle="--")
ax.legend();

### Model Metrics

These features were fed into an ordinary least-squares multiple regression model. This model:

* is statistically significant overall (F-statistic p-value 5.53e-134)
* explains about 82% of the variance in MPG (adjusted R-Squared 0.824)
* is off by about 3.2 MPG in an average prediction (RMSE 3.2005653137923917)

### Model Diagnostics

Below are partial regression plots for all model features:

In [ ]:
fig = plt.figure(figsize=(15,15))
sm.graphics.plot_partregress_grid(
    results,
    exog_idx=list(X.columns),
    grid=(4,4),
    fig=fig)
plt.tight_layout()
plt.show()

### Model Interpretation

Below are all model coefficients and p-values:

In [ ]:
results_df = pd.concat([results.params, results.pvalues], axis=1)
results_df.columns = ["coefficient", "p-value"]
results_df

And these are only coefficients with p-values below 0.05:

In [ ]:
results_df = results_df[results_df["p-value"] < 0.05].sort_values(by="coefficient")
results_df

Writing this out:

* For a car with `weight` and `model year` of 0 (i.e. weighing 0 lbs and built in 1900), as well as a `make` of `amc`, we would expect an MPG of about -17.6
  * It's ok that this is not a valid MPG, since a car weighing 0 lbs is not valid either
* For each increase of 1 lb in **weight**, we see an associated **decrease** in MPG of about .006
* For each year **newer** of a model, we see an associated **increase** in MPG of about 0.75
* Breaking down the `make`:
  * For a **Plymouth** vehicle compared to an AMC vehicle, we see an associated **increase** in MPG of about 2.4
  * For a **Toyota** vehicle compared to an AMC vehicle, we see an associated **increase** in MPG of about 2.5
  * For a **Pontiac** vehicle compared to an AMC vehicle, we see an associated **increase** in MPG of about 3.0
  * For a **Volkswagen** vehicle compared to an AMC vehicle, we see an associated **increase** in MPG of about 4.3
  * For a **Honda** vehicle compared to an AMC vehicle, we see an associated **increase** in MPG of about 4.5
  * For a **Datsun** vehicle compared to an AMC vehicle, we see an associated **increase** in MPG of about 4.7

In [ ]:
results_df.drop(["const", "weight", "model year", "make_other"]).plot.barh(y="coefficient");

Note that while we included `make` categories with higher p-values in the model, we did not report their coefficients. That is because we can't be confident that those coefficients aren't actually zero. Also we did not report a coefficient for `make_other` because this is not a particularly meaningful thing to explain to a stakeholder.

We also aren't plotting the `make` values on the same graph as the `weight` and `model year` values because they are all measured in different units (lbs vs. years vs. vehicle make), so it doesn't really make sense to compare them. Whereas it might be interesting to a stakeholder to see just the `make` values compared against each other.

## Summary

In this lesson you looked at several different multiple regression models, all built using one-hot encoded categorical features in addition to numeric features. You saw how the choice of a reference category impacts the interpretation of the coefficients, and also walked through several different examples of what to do and what not to do when you encounter a feature with many categories.